# Anyscale docs | RAG-based LLM chatbot

This notebook builds a local chatbot that answers questions about Anyscale and Ray by pulling relevant context from docs.anyscale.com and docs.ray.io. All documents are embedded with Open AI's `text-embedding-3-large` and queries route through `gpt-4o`.

* **Organization:** anyscale-internal (staging)
* **Cloud:** `anyscale_v2_default_cloud`
* **Region:** AWS `us-west-2`
* **Compute config:** `g3.8xlarge` head node, which has 2 GPUs and 32 CPUs
* **Base image:** `anyscale/ray:2.35.0-py312`
* **GitHub repository (forked from ray-project/llm-applications):** https://github.com/emmyscode/anyscale-ragbot
* **Workspace:** https://console.anyscale-staging.com/v2/cld_kvedZWag2qA8i5BjxUevf5i7/prj_xQEUDtTyHnTLLx77QG9jRvWY/workspaces/expwrk_9n7tvbjpl7zwhwu33s7r683phe/ses_vig1su7dbnvzqbs3l1tw35fhk3?workspace-tab=code

## 0. Initialize the environment

Once you have the workspace up, complete the following:  

1. Clone the repository
    - `git clone https://github.com/emmyscode/anyscale-ragbot.git` 
2. Install dependencies
    - `pip install -r requirements.txt`
    - `export PYTHONPATH=$PYTHONPATH:$PWD`
    - `pre-commit install`
    - `pre-commit autoupdate`
3. Set up credentials
    - `touch .env`
    - Add in `OPENAI_API_BASE`, `OPENAI_API_KEY`, and `DB_CONNECTION_STRING` to the `.env` file
    - `source .env`

In [ ]:
import os
import ray
import sys; sys.path.append("..")
import warnings; warnings.filterwarnings("ignore")

from dotenv import load_dotenv; load_dotenv()
from rag.config import EMBEDDING_DIMENSIONS, MAX_CONTEXT_LENGTHS, ROOT_DIR

%load_ext autoreload
%autoreload 2

In [ ]:
# Start the Ray cluster, with relevant credentials.

ray.init(runtime_env={
    "env_vars": {
        "OPENAI_API_BASE": os.environ["OPENAI_API_BASE"],
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"], 
        "DB_CONNECTION_STRING": os.environ["DB_CONNECTION_STRING"],
    },
    "working_dir": str(ROOT_DIR)
})

## Data

I've pre-loaded the data into `/mnt/shared_storage/emmy` for both Ray docs (`/mnt/shared_storage/emmy/docs.ray.io/en/master`) and Anyscale docs (`/mnt/shared_storage/emmy/docs.anyscale.com/docs`) respectively. So in this section, we'll clean and chunk the data.

In [ ]:
from pathlib import Path
from rag.config import EFS_DIR

ANYSCALE_DOCS_DIR = Path(EFS_DIR, "docs.anyscale.com/docs")
ANYSCALE_DOCS_URL = "https://docs.anyscale.com"

### This part creates the Anyscale sections

In [ ]:
# Create a list of dictionaries, each containing the source and text
data = []
for path in ANYSCALE_DOCS_DIR.rglob("*.md"):
    if not path.is_dir():
        with open(path, 'r', encoding='utf-8') as file:
            text = file.read()
        # Convert the file path to a URL, remove the '.md' extension
        relative_path = path.relative_to(ANYSCALE_DOCS_DIR).with_suffix('')  # Remove the '.md'
        source = f"{ANYSCALE_DOCS_URL}/{relative_path.as_posix()}"
        data.append({"source": source, "text": text})

In [ ]:
anyscale_sections_ds = ray.data.from_items(data)

### This part creates the Ray sections

In [ ]:
from rag.data import extract_sections

In [ ]:
RAY_DOCS_DIR = Path(EFS_DIR, "docs.ray.io/en/master")
ray_ds = ray.data.from_items([{"path": path} for path in RAY_DOCS_DIR.rglob("*.html") if not path.is_dir()])

In [ ]:
ray_sections_ds = ray_ds.flat_map(extract_sections)

## Chunking

### Anyscale md chunking first

In [ ]:
from functools import partial
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [ ]:
def chunk_md(md_doc):
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, 
        strip_headers=False
        )
    
    chunks = markdown_splitter.split_text(md_doc["text"])
    return[{"text": chunk.page_content, "source": md_doc["source"]} for chunk in chunks]


In [ ]:
chunks_ds = anyscale_sections_ds.flat_map(chunk_md)

### Then Ray chunking

In [ ]:
def chunk_section(section, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len)
    chunks = text_splitter.create_documents(
        texts=[section["text"]], 
        metadatas=[{"source": section["source"]}])
    return [{"text": chunk.page_content, "source": chunk.metadata["source"]} for chunk in chunks]

In [ ]:
ray_chunks_ds = ray_sections_ds.flat_map(partial(
    chunk_section,
    chunk_size=300,
    chunk_overlap=50
))

## Embed

To simplify, I'm just doing OpenAI across the board.

In [ ]:
from langchain_openai import OpenAIEmbeddings

### Anyscale first

In [ ]:
embedding_model = OpenAIEmbeddings(
            model="text-embedding-3-large",
            openai_api_base=os.environ["OPENAI_API_BASE"],
            openai_api_key=os.environ["OPENAI_API_KEY"]
            )

In [ ]:
class EmbedChunks:
    def __init__(self):
        self.embedding_model = OpenAIEmbeddings(
            model="text-embedding-3-large",
            openai_api_base=os.environ["OPENAI_API_BASE"],
            openai_api_key=os.environ["OPENAI_API_KEY"]
            )
    def __call__(self, batch):
        embeddings = self.embedding_model.embed_documents(batch["text"])
        return {"text": batch["text"], "source": batch["source"], "embeddings": embeddings}

In [ ]:
embedded_chunks = chunks_ds.map_batches(
    EmbedChunks,
    batch_size=100, 
    num_gpus=1,
    concurrency=1)

### Ray next

In [ ]:
ray_embedded_chunks = ray_chunks_ds.map_batches(
    EmbedChunks,
    batch_size=100, 
    num_gpus=1,
    concurrency=1)

## Store vectors

In [ ]:
import psycopg
from pgvector.psycopg import register_vector

embedding_model_name = "text-embedding-3-large"

os.environ["MIGRATION_FP"] = f"../migrations/vector-{EMBEDDING_DIMENSIONS[embedding_model_name]}.sql"
os.environ["SQL_DUMP_FP"] = f"{EFS_DIR}/sql_dumps/{embedding_model_name.split('/')[-1]}.sql"

In [ ]:
%%bash
# Set up
psql "$DB_CONNECTION_STRING" -c "DROP TABLE IF EXISTS document;"
echo $MIGRATION_FP
sudo -u postgres psql -f $MIGRATION_FP
echo $SQL_DUMP_FP

In [ ]:
%%bash
# Drop the existing `document` table and create a new one with the schema to store embeddings. 
psql "$DB_CONNECTION_STRING" -c "DROP TABLE IF EXISTS document;"  # drop
sudo -u postgres psql -f $MIGRATION_FP  # create
psql "$DB_CONNECTION_STRING" -c "SELECT count(*) FROM document;"  # num rows

# DROP TABLE
# CREATE TABLE
#  count 
# -------
#      0
# (1 row)

In [ ]:
class StoreResults:
    def __call__(self, batch):
        with psycopg.connect(os.environ["DB_CONNECTION_STRING"]) as conn:
            register_vector(conn)
            with conn.cursor() as cur:
                for text, source, embedding in zip(batch["text"], batch["source"], batch["embeddings"]):
                    cur.execute("INSERT INTO document (text, source, embedding) VALUES (%s, %s, %s)", (text, source, embedding,),)
        return {}

In [ ]:
# Index Anyscale docs
embedded_chunks.map_batches(
    StoreResults,
    batch_size=128,
    num_cpus=1,
    concurrency=6,
).materialize()

# Verify whether the embedding was stored successfully in Postgres or not.
# sudo -u postgres psql
# SELECT * FROM document LIMIT 10;

In [ ]:
# Index Ray docs
ray_embedded_chunks.map_batches(
    StoreResults,
    batch_size=128,
    num_cpus=1,
    concurrency=6,
).materialize()

# Verify whether the embedding was stored successfully in Postgres or not.
# sudo -u postgres psql
# SELECT * FROM document LIMIT 10;

In [ ]:
%%bash
# Save index
rm -rf $SQL_DUMP_FP
mkdir -p $(dirname "$SQL_DUMP_FP") && touch $SQL_DUMP_FP
sudo -u postgres pg_dump -c > $SQL_DUMP_FP  # save

## Retrieval

In [ ]:
import json
import numpy as np

In [ ]:
# Embed query
embedding_model = OpenAIEmbeddings(model=embedding_model_name)
query = "What are the different kinds of storage, and how do I use them?"
embedding = np.array(embedding_model.embed_query(query))
len(embedding)

In [ ]:
# Get context
num_chunks = 10
with psycopg.connect(os.environ["DB_CONNECTION_STRING"]) as conn:
    register_vector(conn)
    with conn.cursor() as cur:
        # cur.execute("SELECT * FROM document ORDER BY embedding <=> %s LIMIT %s", (embedding, num_chunks))
        cur.execute("SELECT *, (embedding <=> %s) AS similarity_score FROM document ORDER BY similarity_score LIMIT %s", (embedding, num_chunks))
        rows = cur.fetchall()
        ids = [row[0] for row in rows]
        context = [{"text": row[1]} for row in rows]
        sources = [row[2] for row in rows]
        scores = [row[4] for row in rows]

In [ ]:
for i, item in enumerate(context):
    print (ids[i])
    print (scores[i])
    print (sources[i])
    print (item["text"])
    print ()

In [ ]:
def semantic_search(query, embedding_model, k):
    embedding = np.array(embedding_model.embed_query(query))
    with psycopg.connect(os.environ["DB_CONNECTION_STRING"]) as conn:
        register_vector(conn)
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM document ORDER BY embedding <=> %s LIMIT %s", (embedding, k),)
            rows = cur.fetchall()
            semantic_context = [{"id": row[0], "text": row[1], "source": row[2]} for row in rows]
    return semantic_context

## Generation

In [ ]:
import openai
import time

In [ ]:
from rag.generate import prepare_response


In [ ]:
from rag.utils import get_client

In [ ]:
def generate_response(
    llm, temperature=0.0, stream=True,
    system_content="", assistant_content="", user_content="", 
    max_retries=1, retry_interval=60):
    """Generate response from an LLM."""
    retry_count = 0
    client = get_client(llm=llm)
    messages = [{"role": role, "content": content} for role, content in [
        ("system", system_content), 
        ("assistant", assistant_content), 
        ("user", user_content)] if content]
    while retry_count <= max_retries:
        try:
            chat_completion = client.chat.completions.create(
                model=llm,
                temperature=temperature,
                stream=stream,
                messages=messages,
            )
            return prepare_response(chat_completion, stream=stream)

        except Exception as e:
            print(f"Exception: {e}")
            time.sleep(retry_interval)  # default is per-minute rate limits
            retry_count += 1
    return ""

In [ ]:
context_results = semantic_search(query=query, embedding_model=embedding_model, k=5)
context = [item["text"] for item in context_results]
print(context)

In [ ]:
# Generate response
query = "What are the different kinds of storage, and how do I use them?"
response = generate_response(
    llm="gpt-4o",
    temperature=0.0,
    stream=True,
    system_content="Answer the query using the context provided. Be succinct.",
    user_content=f"query: {query}, context: {context}")
# Stream response
for content in response:
    print(content, end='', flush=True)

## Agent

In [ ]:
from rag.embed import get_embedding_model
from rag.utils import get_num_tokens, trim

In [ ]:
class QueryAgent:
    def __init__(self, embedding_model_name="text-embedding-3-large",
                 llm="gpt-4o", temperature=0.0, 
                 max_context_length=4096, system_content="", assistant_content=""):
        
        # Embedding model
        self.embedding_model = OpenAIEmbeddings(
            model="text-embedding-3-large",
            openai_api_base=os.environ["OPENAI_API_BASE"],
            openai_api_key=os.environ["OPENAI_API_KEY"]
            )
        
        # Context length (restrict input length to 50% of total context length)
        max_context_length = int(0.5*max_context_length)
        
        # LLM
        self.llm = llm
        self.temperature = temperature
        self.context_length = max_context_length - get_num_tokens(system_content + assistant_content)
        self.system_content = system_content
        self.assistant_content = assistant_content

    def __call__(self, query, num_chunks=5, stream=True):
        # Get sources and context
        context_results = semantic_search(
            query=query, 
            embedding_model=self.embedding_model, 
            k=num_chunks)
            
        # Generate response
        context = [item["text"] for item in context_results]
        sources = [item["source"] for item in context_results]
        user_content = f"query: {query}, context: {context}"
        answer = generate_response(
            llm=self.llm,
            temperature=self.temperature,
            stream=stream,
            system_content=self.system_content,
            assistant_content=self.assistant_content,
            user_content=trim(user_content, self.context_length))

        # Result
        result = {
            "question": query,
            "sources": sources,
            "answer": answer,
            "llm": self.llm,
        }
        return result

In [ ]:
embedding_model_name = "text-embedding-3-large"
llm = "gpt-4o"

In [ ]:
query = "What happens when I duplicate a workspace?"
system_content = "Answer the query using the context provided. Be succinct."
agent = QueryAgent(
    embedding_model_name=embedding_model_name,
    llm=llm,
    max_context_length=MAX_CONTEXT_LENGTHS[llm],
    system_content=system_content)
result = agent(query=query, stream=False)
print(json.dumps(result, indent=2))